# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [4]:
import pandas as pd
import numpy as np
import math
import json
import boto3
import sagemaker
from sklearn.model_selection import train_test_split
%matplotlib inline

In [5]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [8]:
transcript.shape

(306534, 4)

In [9]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [10]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [11]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [12]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [15]:
df = transcript.copy()
df.head()


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [13]:
# make profile id the index
profile_id_as_index = profile.copy()
profile_id_as_index.set_index('id', inplace=True)
profile_id_as_index

,gender,age,became_member_on,income
id,,,,
68be06ca386d4c31939f3a4f0e3dd783,None,118,20170212,NaN
0610b486422d4921ae7d2bf64640c50b,F,55,20170715,112000.0
38fe809add3b4fcf9315a9694bb96ff5,None,118,20180712,NaN
78afa995795e4d85b5d9ceeca43f5fef,F,75,20170509,100000.0
a03223e636434f42ac4c3df47e8bac43,None,118,20170804,NaN
...,...,...,...,...
6d5f3a774f3d4714ab0c092238f3a1d7,F,45,20180604,54000.0
2cb4f97358b841b9a9773a7aa05a9d77,M,61,20180713,72000.0
01d26f638c274aa0b965d24cefe3183f,M,49,20170126,73000.0


In [17]:
# Filter out the customer who have completed the offers.
completed_offers = df[df['event'] == 'offer completed'].copy()
completed_offers['offer_id'] = completed_offers.apply(lambda row: row.value['offer_id'], axis=1)
completed_offers



,person,event,value,time,offer_id
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,2906b810c7d4411798c6938adc9daaa5
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0,fafdcd668e3743c1bb461111dcafc2a4
12679,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
12692,676506bad68e4161b9bbaffeb039626b,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,0,ae264e3637204a6fb9bb56bc8210ddfd
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,714,2298d6c36e964ae4a3e7e9706d1fb8c2
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,714,2298d6c36e964ae4a3e7e9706d1fb8c2
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714,fafdcd668e3743c1bb461111dcafc2a4
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714,fafdcd668e3743c1bb461111dcafc2a4


In [34]:
# Filer out all the customers who have viewed the offer.
viewed_offers_df = df[df['event'] == 'offer viewed'].iloc[0:1000].copy()
viewed_offers_df

,person,event,value,time
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
12652,102e9454054946fda62242d2e176fdce,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
12653,02c083884c7d45b39cc68e1314fec56c,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0
12655,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
...,...,...,...,...
14048,7f370edef8ae471bb1e3067f84f5a9d0,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
14049,a9c89c0882154db68a99adcf81eac27e,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
14050,798d915dfb6a4477adf06802ba29477e,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0
14051,e4c38c75c12f4bec9fb35546b14c5fd5,offer viewed,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0


In [21]:
# method will find out all the cutomers whoi have completed the offers.
def is_offer_used(person_id, offer_id, completed_offers):
    offer_used = completed_offers[(completed_offers['person'] == person_id) & (completed_offers['offer_id'] == offer_id)]
    return int(not offer_used.empty)

In [35]:
# Label 1 if a customer view the offer and also completed the offer.
# Label 0 if a customer view the offer but did not complete the offer.
viewed_offers_df['offer_used_label'] = viewed_offers_df.apply(lambda row: is_offer_used(row['person'], row['value']['offer id'], completed_offers), axis=1)
viewed_offers_df

,person,event,value,time,offer_used_label
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0,1
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0,0
12652,102e9454054946fda62242d2e176fdce,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1
12653,02c083884c7d45b39cc68e1314fec56c,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0,0
12655,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0,0
...,...,...,...,...,...
14048,7f370edef8ae471bb1e3067f84f5a9d0,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0
14049,a9c89c0882154db68a99adcf81eac27e,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0
14050,798d915dfb6a4477adf06802ba29477e,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0,1
14051,e4c38c75c12f4bec9fb35546b14c5fd5,offer viewed,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1


In [36]:
# total offers viewed but not completed
viewed_offers_df[viewed_offers_df['offer_used_label'] == 0].count()

person              502
event               502
value               502
time                502
offer_used_label    502
dtype: int64

In [37]:
# total offers viewed and completed
viewed_offers_df[viewed_offers_df['offer_used_label'] == 1].count()

# From above 2 cells we find that data set is balanced as 
# count for label 0 = 28269
# count for label 1 = 29456

person              498
event               498
value               498
time                498
offer_used_label    498
dtype: int64

In [39]:
# create valid offers and enrich with extra info.
valid_offers_df = pd.DataFrame(columns=['age', 'became_member_on', 'gender', 'income', 'event', 'person', 'time', 'value', 'offer_used_label'])
for idx, viewed_offer in viewed_offers_df.iterrows():
    valid_offers_df.loc[idx] = profile_id_as_index.loc[viewed_offer['person']].append(viewed_offer)
valid_offers_df

,age,became_member_on,gender,income,event,person,time,value,offer_used_label
12650,65,20180209,M,53000.0,offer viewed,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},1
12651,53,20170916,O,52000.0,offer viewed,d1ede868e29245ea91818a903fec04c6,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
12652,69,20160814,F,57000.0,offer viewed,102e9454054946fda62242d2e176fdce,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1
12653,20,20160711,F,30000.0,offer viewed,02c083884c7d45b39cc68e1314fec56c,0,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0
12655,39,20140527,M,51000.0,offer viewed,be8a5d1981a2458d90b255ddc7e0d174,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
...,...,...,...,...,...,...,...,...,...
14048,53,20141201,M,100000.0,offer viewed,7f370edef8ae471bb1e3067f84f5a9d0,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
14049,118,20180501,None,NaN,offer viewed,a9c89c0882154db68a99adcf81eac27e,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
14050,78,20131229,M,57000.0,offer viewed,798d915dfb6a4477adf06802ba29477e,0,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},1
14051,64,20170703,M,98000.0,offer viewed,e4c38c75c12f4bec9fb35546b14c5fd5,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},1


In [61]:
# fill 0 for income where its not available
valid_offers_df['income'].fillna(0, inplace=True)
valid_offers_df.count()

age                 1000
became_member_on    1000
gender               840
income              1000
event               1000
person              1000
time                1000
value               1000
offer_used_label    1000
dtype: int64

In [62]:
# Temporarily drop non-converted features
valid_offers_df.drop(['became_member_on', 'gender', 'event', 'person', 'value'], axis=1, inplace=True)


In [63]:
valid_offers_df

,age,income,time,offer_used_label
12650,65,53000.0,0,1
12651,53,52000.0,0,0
12652,69,57000.0,0,1
12653,20,30000.0,0,0
12655,39,51000.0,0,0
...,...,...,...,...
14048,53,100000.0,0,0
14049,118,0.0,0,0
14050,78,57000.0,0,1
14051,64,98000.0,0,1


In [64]:
# Split data into labels and features
#valid_offers_used = valid_offers_df['offer_used_label'].copy()
valid_offers_used_label = valid_offers_df['offer_used_label'].copy()
valid_offers_used_features = valid_offers_df.drop(['offer_used_label'], axis=1)

In [65]:

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(valid_offers_used_features, 
                                                    valid_offers_used_label, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [66]:
X_train.count()
y_train.head()

X_test.head()
y_test.head()

14044    0
13843    1
13051    0
13391    1
13567    0
Name: offer_used_label, dtype: object

## Upload Data to S3

In [67]:
# upload data to s3
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-eu-north-1-186768347586'

In [68]:
import os

def make_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # combine labels and features 
    pd.concat([y, x], axis=1)\
        .to_csv(os.path.join(data_dir, filename), header=False, index=False)
    
    # indicate function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [69]:
data_dir = 'data_starbucks' # the folder I will use for storing data in notebook
name = 'train.csv'

# create 'train.csv'
make_csv(X_train, y_train, name, data_dir)

Path created: data_starbucks/train.csv


In [70]:
# directory for features data
data_dir = 'data_starbucks'

# prefix: description name for directory in S3
prefix = 'starbucks-capstone/offer-data'

# upload all data to S3
uploaded_data_s3 = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(uploaded_data_s3)

s3://sagemaker-eu-north-1-186768347586/starbucks-capstone/offer-data


In [71]:
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

sagemaker-scikit-learn-2021-04-11-08-22-07-540/profiler-output/system/incremental/2021041108/1618129440.algo-1.json
sagemaker-scikit-learn-2021-04-11-08-22-07-540/source/sourcedir.tar.gz
sagemaker-scikit-learn-2021-04-11-08-35-46-311/profiler-output/system/incremental/2021041108/1618130220.algo-1.json
sagemaker-scikit-learn-2021-04-11-08-35-46-311/profiler-output/system/incremental/2021041108/1618130280.algo-1.json
sagemaker-scikit-learn-2021-04-11-08-35-46-311/rule-output/ProfilerReport-1618130146/profiler-output/profiler-report.html
sagemaker-scikit-learn-2021-04-11-08-35-46-311/rule-output/ProfilerReport-1618130146/profiler-output/profiler-report.ipynb
sagemaker-scikit-learn-2021-04-11-08-35-46-311/rule-output/ProfilerReport-1618130146/profiler-output/profiler-reports/BatchSize.json
sagemaker-scikit-learn-2021-04-11-08-35-46-311/rule-output/ProfilerReport-1618130146/profiler-output/profiler-reports/CPUBottleneck.json
sagemaker-scikit-learn-2021-04-11-08-35-46-311/rule-output/Profile

## 1. SKLearn: LinearSVC

In [72]:
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

# sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. 
# from sklearn.externals import joblib
# Import joblib package directly
import joblib

## TODO: Import any additional libraries you need to define a model
from sklearn.svm import LinearSVC


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argp

## Define and train model

In [73]:
# Define and train model
from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point='train.py',
                    source_dir='/home/ec2-user/SageMaker/Capstone-Starbucks-Project/source_sklearn',
                    role=role,
                    py_version="py3",
                    framework_version='0.23-1',
                    train_instance_count=1,
                    train_instance_type='ml.m5.large')

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Train

In [74]:
%time
# Train estimator on S3 training data
estimator.fit({'train': uploaded_data_s3})

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
2021-04-11 08:50:00 Starting - Starting the training job...
2021-04-11 08:50:27 Starting - Launching requested ML instancesProfilerReport-1618130999: InProgress
......
2021-04-11 08:51:28 Starting - Preparing the instances for training......
2021-04-11 08:52:28 Downloading - Downloading input data...
2021-04-11 08:52:59 Training - Training image download completed. Training in progress..2021-04-11 08:53:00,253 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-04-11 08:53:00,261 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-11 08:53:00,276 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-04-11 08:53:00,596 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-11 08:53:03,620 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-11 08:

## Deploy

In [75]:

%%time

# deploy model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large')

-------------!CPU times: user 205 ms, sys: 15.9 ms, total: 221 ms
Wall time: 6min 32s


In [76]:

# Determine accuracy of model
# First: generate predicted, class labels
test_y_preds = predictor.predict(X_test)

In [77]:
# test that model generates the correct number of labels
assert len(test_y_preds)==len(y_test), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [79]:
test_y = pd.DataFrame(y_test).iloc[:,0]
converted_test_y = test_y.reset_index(drop=True).values.astype(int)
converted_test_y

array([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0])

## Check accuracy

In [80]:
from sklearn.metrics import accuracy_score

# Second: calculate the test accuracy
accuracy = accuracy_score(converted_test_y, test_y_preds.round())

In [81]:

print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.635

Predicted class labels: 
[0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1
 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]

True class labels: 
[0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0
 0 0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1
 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1 0 0
 1 0 1 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1
 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 1 0 1
 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0]


## 2. PyTorch: Feed Forward Neural Network

In [82]:
#PyTorch: Feed Forward Neural Network 
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # Load the parameters used to create the model
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(f))

    # Set to eval mode,

## Create PyTorch Estimator

In [87]:

# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch',
                    role=role,
                    py_version="py3",
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.m5.large',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_features': 3, # number of features
                        'hidden_dim': 2,
                        'output_dim': 1,
                        'epochs': 80
                    })

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Train the Estimator

In [88]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': uploaded_data_s3})

2021-04-11 09:15:35 Starting - Starting the training job...
2021-04-11 09:15:37 Starting - Launching requested ML instancesProfilerReport-1618132534: InProgress
......
2021-04-11 09:16:43 Starting - Preparing the instances for training......
2021-04-11 09:18:05 Downloading - Downloading input data
2021-04-11 09:18:05 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-04-11 09:18:06,410 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-04-11 09:18:06,413 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-04-11 09:18:06,429 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-04-11 09:18:07,058 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-04-11 09:18:07,296 sagemaker-containers INFO     Module train does not p

## Instantiate a PyTorchModel

In [89]:
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role=role,
                     py_version="py3",
                     framework_version='1.0',
                     entry_point='predict.py',
                     source_dir='source_pytorch')

## Deploy Model

In [91]:
%%time
# deploy and create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

-----------!CPU times: user 212 ms, sys: 10.7 ms, total: 223 ms
Wall time: 5min 31s


In [92]:
# Convert test data
X_test_converted = X_test.reset_index(drop=True).values.astype(int)
y_test_converted = test_y.reset_index(drop=True).values.astype(int)

In [93]:
# Evaluate the model
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # rounding and squeezing array
    test_preds = np.squeeze(np.round(predictor.predict(test_features)))
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

In [94]:
# get metrics for custom predictor
metrics = evaluate(predictor, X_test_converted, y_test_converted, True)

predictions  0.0  1.0
actuals              
0             33   71
1              4   92

Recall:     0.958
Precision:  0.564
Accuracy:   0.625



## Delete Endpoint

In [ ]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [ ]:
# delete the predictor endpoint 
delete_endpoint(predictor)